In [1]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

In [2]:
data = pd.read_csv('AVANCE_6.csv', dtype={
    'COD_UNICO': str,
    'PERIODO': str
})
data.head(3)

,_6_cut_up_F_Septile,ADEX_PROM_U6M_ACTUAL_IMP_OVER_12M_PASADO_MENOR_0.55,_aux_TRX_CAIDA_5PERCENT_COBRANZAS__MES_ANT,ACTIVIDAD,_300_cut_up_ROTACION_INVENTARIOS_CEF_SITUACION,_600000.0_cut_up_RIESGO,TRX_SUBIDA_SEGUIDA_U3M_CASH_IN,TIENE_DISMINUCION__CASHIN_50PERCENT_A_MAS_U12M,MAX_DIAS_ATRASO_BCPSF_ULT_RCC_x,TIENE_PRESTAMOS_Y_LEASING_VIGENTES_SF_ULT_RCC,...,_3_cut_up_NUM_DOCS_PROT_PROMEDIO_U9M,_2_cut_up_NUM_DOCS_PROT_U6M,_2_cut_up_NUM_DOCS_LAB_U1M,_0.4_cut_up_PERCENT_GAR_OTROS_BANCOS_PRIN,_5_cut_up_MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x,_0.4_cut_down_minimo_percent_auto_hipo_u12m,departamento_preocupa,_1_cut_up_NUM_DOCS_TRIB_U6M,_0.5_cut_down_SOW_OTROSBCOS_MAX_historia,_1_cut_down_NUM_BANCOS_DEBT_MAYOR_DE_U3M
0,0.0,0.0,0.0,NaN,1.0,0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0,0.0,0.0,1
1,0.0,0.0,0.0,Comercio,1.0,0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1,0.0,0.0,0
2,0.0,0.0,0.0,NaN,0.0,1,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0,0.0,0.0,1


In [3]:
from utils_alertas import *

print(data.shape)
data = filtro_banca_mediana(data)
print(data.shape)
#col_target = target_severo

pd.crosstab(
    data[col_target].fillna(-1), data['PERIODO']
)

EJEMPLO:
202104 mes cerrado de la cosecha de 202104 [202011, 202012, 202101, 202102, 202103, 202104] [202102, 202103, 202104] 
 historia 
 [202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104] 
 ---------------------------------------------------------------------------------------------------- 
 [202105, 202106] 
 ---------------------------------------------------------------------------------------------------- 
 [202005, 202006]
EJEMPLO:
202003 mes cerrado de la cosecha de 202003 [201910, 201911, 201912, 202001, 202002, 202003] [202001, 202002, 202003] 
 historia 
 [201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912, 202001, 202002, 202003] 
 ---------------------------------------------------------------------------------------------------- 
 [202004, 202005] 
 ---------------------------------------------------------------------------------------------------- 
 [201904, 201905]
(24014, 4042)
(4829, 4042)


PERIODO,202009,202010,202011,202012,202101,202102
target_FINAL_fusion_atraso_feve_2Mbueno,,,,,,
-1.0,269,239,221,204,197,193
0.0,515,519,522,552,554,553
1.0,50,49,45,47,48,52


In [4]:
data.shape, data[data[col_target].notnull()].shape
data = data[data[col_target].notnull()]

print(data.shape)

(3506, 4042)


In [5]:
data = data[data['PERIODO'].isin(sorted(data['PERIODO'].unique())[-2:])]
data['PERIODO'].value_counts(dropna=False)

202102    605
202101    602
Name: PERIODO, dtype: int64

In [6]:
malos = data[
    (data[col_target] == 1 ) #\
    #   & (setalertas['TIENE_DEUDA_REFINANCIADO_RCC'] == 0) & (setalertas['TIENE_DEUDA_REFINANCIADO_IBK'] == 0) & \
    #(setalertas['_3_cut_up_NUM_PROD_DEUDA_VIGENTE_SF_ULT_RCC'] == 0)
]
total_malos = len(
    set(malos['COD_UNICO'])
)
total_cartera = len(
    set(
        data['COD_UNICO']
    )
)

print("malos detectados 2M adelante: ", total_malos, ", total cartera buenos actuales", total_cartera) 

malos detectados 2M adelante:  77 , total cartera buenos actuales 643


#### COMBINACIONES DE 3 ---- 3 COSECHAS

In [7]:
_vars = pd.read_csv('COMBINACIONES_MEDIANA_LIMA.csv', sep=',').sort_values(
    by=['SCORE'], ascending=False
).drop_duplicates(
    subset=['VAR_1', 'VAR_2', 'VAR_3']
)
_vars['ALL_VARS'] = _vars['VAR_1'].astype(str) + _vars['VAR_2'].astype(str) + _vars['VAR_3'].astype(str)

print(_vars.shape)
_vars.head(55)

(18179, 7)


,VAR_1,VAR_2,VAR_3,CASOS,CORRECTOS,SCORE,ALL_VARS
16729,_4_cut_up_RMET_9_METODIZADO_CEF_SITUACION,_1_cut_up_NUM_DOCS_LAB_U12M,NaN,6,6.0,1.000000,_4_cut_up_RMET_9_METODIZADO_CEF_SITUACION_1_cu...
11985,_3_cut_up_NUM_MESES_ANTERIORES_CON_IMPORTE_LAB...,_aux_TRX_CAIDA_30PERCENT_OPERACIONES_MESA__U11M,NaN,5,5.0,1.000000,_3_cut_up_NUM_MESES_ANTERIORES_CON_IMPORTE_LAB...
12710,_4_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF,_0_cut_up_TIENE_TENDENCIA_CRECIENTE_IMPORTE_IM...,NaN,6,6.0,1.000000,_4_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_S...
17071,_9_cut_up_NUM_MESES_ANTERIORES_CON_IMPORTE_IMP...,_2_cut_up_MAX_DIAS_ATRASO_SFSF_ULT_RCC,NaN,7,7.0,1.000000,_9_cut_up_NUM_MESES_ANTERIORES_CON_IMPORTE_IMP...
6352,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.2,DEBT_CAIDA_SEGUIDA_MTOSALDO_IBK_U3M_SF,NaN,5,5.0,1.000000,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1....
16298,TIENE_IMPORTE_TRIB_REGISTRADO_MES_ACTUAL,_7_cut_up_MAX_DIAS_ATRASO_SF,NaN,7,7.0,1.000000,TIENE_IMPORTE_TRIB_REGISTRADO_MES_ACTUAL_7_cut...
7303,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.3,_0_cut_up_TENDENCIA_NUM_DOCS_TRIB_U12M,NaN,7,7.0,1.000000,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1....
16020,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.3,_0_cut_up_TENDENCIA_NUM_DOCS_TRIB_U12M,NaN,7,7.0,1.000000,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.3...
16056,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.2,TIENE_TENDENCIA_CRECIENTE_NRO_DOCS_TRIB_U12M,NaN,7,7.0,1.000000,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1....
2231,_2_cut_up_MAX_DIAS_ATRASO_SFSF_ULT_RCC,_9_cut_up_NUM_MESES_ANTERIORES_CON_IMPORTE_IMP...,NaN,7,7.0,1.000000,_2_cut_up_MAX_DIAS_ATRASO_SFSF_ULT_RCC_9_cut_u...


In [8]:
select = _vars[
    (~_vars['ALL_VARS'].astype(str).str.contains('COVID'))
].sort_values(by=['SCORE'], ascending=False)

select.shape

(18179, 7)

#### COMBINACIONES DE 2 ---- 3 COSECHAS

In [9]:
data['PERIODO'].value_counts()

202102    605
202101    602
Name: PERIODO, dtype: int64

In [10]:
data['ZONAL'].value_counts()

BEL ZONAL 2    1207
Name: ZONAL, dtype: int64

In [11]:
data['SEGMENTO'].value_counts()

S2    1207
Name: SEGMENTO, dtype: int64

In [12]:
#[col for col in data.columns if 'NORMAL' in col.upper()]

In [13]:
data['NO_CLASIF_NORMAL'] = data['IS_NORMAL_IBK_SBS'].apply(lambda _: not(_)).astype(int)

pd.crosstab(
    data['NO_CLASIF_NORMAL'], data['IS_NORMAL_IBK_SBS']
)

IS_NORMAL_IBK_SBS,0,1
NO_CLASIF_NORMAL,,
0,0,1195
1,12,0


In [14]:
for colum in ['VAR_1', 'VAR_2', 'VAR_3']:
    try:
        select[colum] = select[colum].apply(
            lambda _: 'NO_CLASIF_NORMAL' if _ in ['CLASIF_SBS__DEFICIENTE', 'CLASIF_SBS__DUDOSO', 'CLASIF_SBS__P.POTENCIAL'] else _)
    except:
        pass
    
    try:
        select_2[colum] = select_2[colum].apply(
            lambda _: 'NO_CLASIF_NORMAL' if _ in ['CLASIF_SBS__DEFICIENTE', 'CLASIF_SBS__DUDOSO', 'CLASIF_SBS__P.POTENCIAL'] else _)  
    except:
        pass

In [15]:
data['TIENE_DEUDA_REFINANCIADO_RCC'].value_counts(dropna=False)

0.0    1207
Name: TIENE_DEUDA_REFINANCIADO_RCC, dtype: int64

In [16]:
[col for col in data.columns if 'fusion' in col.lower()]

['target_FINAL_fusion_atraso_SEGUIDO_15000_FEVE_2Mbueno',
 'fusion_atraso_grantia',
 'target_fusion_atraso_feve_3_1Mbueno',
 'target_fusion_atraso_feve_3_3Mbueno',
 'target_fusion_atraso_15_feve_all',
 'target_fusion_atraso_15_feve_all_2Mbueno',
 'target_fusion_atraso_feve_3',
 'target_FINAL_fusion_atraso_feve_2Mbueno',
 'target_fusion_atraso_feve',
 'target_fusion_atraso_feve_2']

In [17]:
for row in select.sort_values(by=['SCORE'], ascending=False).iterrows():
    #print(type(row), row[0], row[1])
    #print("/"*10)
    cols_1 = row[1][0]
    cols_2 = row[1][1]
    cols_3 = row[1][2]
    cantidad = row[1][3]
    correctos = row[1][4]
    score_feve = float(row[1][5])
    print("---> COLS: ", cols_1, cols_2, cols_3)
    print("---> CANTIDAD: ", cantidad, correctos, score_feve)
    
    #if pd.isnull(cols_3):
    #    break
    break

---> COLS:  _4_cut_up_RMET_9_METODIZADO_CEF_SITUACION _1_cut_up_NUM_DOCS_LAB_U12M nan
---> CANTIDAD:  6 6.0 1.0


In [18]:
select.shape

(18179, 7)

In [19]:
data.shape, #data

((1207, 4043),)

## >> Descarte de los ya alertados

In [20]:
lista_alertas_orden = [
     #### ############################################### MEDIANAS LIMA #######################################################
  ## A1
    ['TIENE_1_O_MAS_GRUPO_ECON_VPC_SEGUIR', 'subsector_obervado_banca'], 
    ## A2 CEF 29/01/2021
     ['_1_cut_up_NUM_CLASIF_NO_NORMAL_GRUPO_EECON', 'FUNNEL_ALERTA'],     ################# FIX QUIT CEF 50% UP ROTACION INVENTARIOS ----
    ## A3 CEF 29/01/2021
    ['_55_cut_up_ROTACION_CTASXCOBRAR_VAR_DIAS_CEF_SITUACION', '_1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC'],   
    ## A4 ### CEF 29/01/2021
    ['_1_cut_up_ROTACION_CTASXCOBRAR_VAR_CEF_SITUACION','_1_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF', '_3_cut_up_NUM_PROD_DEUDA_VIGENTE_SF_ULT_RCC'],
    ##A5 ### CEF 29/01/2021    
    ['_1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC', '_30_cut_up_ROTACION_INVENTARIOS_VAR_DIAS_CEF_SITUACION'],
    ## A6 CEF 29/01/2021
    ['_0.8_cut_up_ENDEUDAMIENTO_PAS_VEN_VAR_CEF_SITUACION', '_50_cut_up_ROTACION_CTASXPAGAR_VAR_DIAS_CEF_SITUACION'],  
    ## A7 CEF 29/01/2021    
    ['TIENE_1_O_MAS_BANCOS_CON_MAYOR_SOW_ULT_RCC', 'TIENE_DISMINUCION_10PERCENT_A_MAS_CANT_TRX_PROM_U6M_U12M', '_30_cut_up_ROTACION_CTASXPAGAR_VAR_DIAS_CEF_SITUACION'], 
    ## A8 CEF 29/01/2021
    ['_0.8_cut_up_ENDEUDAMIENTO_PAS_VEN_VAR_CEF_SITUACION', '_1_cut_up_MAX_DIAS_ATRASO_SFSF_ULT_RCC', '_0.7_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC'],
    ## A9
    ['_-5_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN', '_1_cut_up_VARACION_NUM_BANCOS_FULL_RCCC_SF'],
    ## A10
    ['TIENE_DISMINUCION_PERSONAL_10PERCENT_A_MAS_U7M','PROTESTOS_REGISTRADO_MAYOR_A_5MIL_SOLES', '_2_cut_up_BUCKET'],
    ## A11
    ['_0.7_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC', 'subsector_obervado_banca', '_1_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF'],  
    ## A12
    ['TIENE_DEUDA_VENCIDO_RCC', 'FUNNEL_ALERTA'], 
    ## A13 CEF FALTAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
    ['_6_cut_up_M_Septile', '_3_cut_up_DTOTAL_S_EBITDA_VAR_CEF_SITUACION'],
    ## A14 CEF 29/01/2021
    ['cut_1_TIENE_DISMINUCION_1_O_MAS_TIPOS_CASH_IN_U3M', 'TIENE_AUMENTO_RIESGO_U7M', '_0.8_cut_up_ENDEUDAMIENTO_PAS_VEN_VAR_CEF_SITUACION'],   
    # A15 CEF 29/01/2021
    ['_0.8_cut_up_ENDEUDAMIENTO_PAS_VEN_VAR_CEF_SITUACION', 'TIENE_DISMINUCION_PERSONAL_10PERCENT_A_MAS_U7M'],  
    ## A16
    ['TIENE_AUMENTO_RIESGO_U7M', 'TIENE_PROTESTOS_REGISTRADO_MES_ACTUAL', '_0_cut_up_TENDENCIA_PROTESTOS_U7M'],
    ## A17
    ['TIENE_DISMINUCION_PERSONAL_10PERCENT_A_MAS_U7M', '_3_cut_up_TRX_TIPOS_CASH_OUT_CAIDA_10PERCENT_MES_ANT'], 
    # A18 CEF 29/01/2021
   ['PROTESTOS_REGISTRADO_MAYOR_A_5MIL_SOLES', 'NO_TIENE_GAR_AUTO_SF', '_1_cut_up_MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x'],  ############### PROPUESTA DE CAMBIO
    #['_aux_TRX_CAIDA_10PERCENT_PAGOS_MASIVOS__CASH_OUT_MES_ANT', '_30_cut_up_ROTACION_CTASXPAGAR_VAR_DIAS_CEF_SITUACION'], # SI
    #['_aux_TRX_CAIDA_10PERCENT_PAGOS_MASIVOS__CASH_OUT_MES_ANT', '_40_cut_up_ROTACION_CTASXPAGAR_VAR_DIAS_CEF_SITUACION'], # SI
    #['_aux_TRX_CAIDA_10PERCENT_PAGOS_MASIVOS__CASH_OUT_MES_ANT', '_50_cut_up_ROTACION_CTASXPAGAR_VAR_DIAS_CEF_SITUACION'], # SI
    
    ## A19
    ['NO_TIENE_TRX_OPERACIONES_MESA__MES_ACTUAL', 'TIENE_AUMENTO_PROMEDIO_DE_DIAS_DE_ATRASO_COLDIRECT_SF','_4_cut_up_F_Quartile'],
    ## A20
    ['TIENE_10PERCENT_O_MAS_GRUPO_ECON_VPC_SEGUIR', 'TIENE_DISMINUCION_10PERCENT_A_MAS_CANT_TRX_CASHOUT_MES_ANT'],
    
    ## A21 #
    #['_0.1_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC', 'TIENE_DISMINUCION_1_O_MAS_TIPOS_CASH_OUT_U3M', '_6_cut_up_M_Septile'],    ################ FIX  0.3 --> 0.1
    ['TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYOR_1', 'TIENE_COVER_IBK_GAR_PREFERIDA_MENOR_0.2', 'TIENE_NRO_DOCS_TRIB_REGISTRADO_MES_ACTUAL'],       ############## SUSTITUCION DE ALERTA
    
    ## A22
    ['_1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC', 'TIENE_AUMENTO_COL_DIRECTA_5PERCENT_A_MAS_U3M'],

    # A23
    ['_1_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF', 'subsector_obervado_banca', 'TIENE_REACTIVA_SF'], ################### FIX TIENE REACTIVA 1 Y 2 ------->  TIENE REACTIVA SF
    #A24
    #['_5_cut_up_NUM_MESES_SIN_LINEA_VIGENTE', 'MAYOR_GARANTIA_Y_CAIDA_10PERCENT_DEBT_COLDIRECT_U3M_NO_IBK_SF'],
    #A25
    #['_1_cut_up_NUM_MESES_ANTERIORES_CON_PROTESTOS_CONSECUTIVOS', 'M_Septile_cut_4.0'],
    
    #A26
    #['TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYORIGUAL_1.3', 'FUNNEL_ALERTA'],
    #['TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYOR_1', '_2_cut_up_NUM_DOCS_PROT_U12M'],
    
    #['TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYOR_1', 'TIENE_NRO_DOCS_PROT_U1M_OVER_U12M_MAYOR_1'],
    #['TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYORIGUAL_1.2', 'FUNNEL_ALERTA'],
    #['TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYORIGUAL_1.1', 'FUNNEL_ALERTA'],
    #['TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYOR_1', 'TIENE_COVER_IBK_GAR_AUTO_HIPO_MENOR_0.2', 'TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1'],
    #['TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYOR_1', '_aux_NO_TIENE_TRX_OPERACIONES_MESA__HACE_12M', 'PROTESTOS_REGISTRADO_MAYOR_A_5MIL_SOLES'],

    
    #['TIENE_REACTIVA_1_2_CON_OTRAS_EF', '_2_cut_up_MAX_DIAS_ATRASO_SFSF_ULT_RCC'],
    #['cut_1_TIENE_AL_MENOS_1_TIPO_CASHIN_ACTIVO_U3M', '_aux_NO_TIENE_CASHOUT_OPERACIONES_MESA__HACE_12M', '_3_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF'],
    #['_3_cut_up_NUM_BANCOS_PRIN_DEUDASF_ULT_RCC', '_10_cut_up_NUM_MESES_ANTERIORES_CON_PROTESTOS_CONSECUTIVOS'],
    #['MORE_0.15_PERCENT_DELAY_OVER_TOTAL_DEBT_CC', 'TIENE_REACTIVA_1_2_CON_OTRAS_EF'],
    #['_3_cut_up_NUM_MESES_ANTERIORES_CON_NUM_DOCS_TRIB_CONSECUTIVOS', 'IMPORTE_TRIB_REGISTRADO_MAYOR_A_20MIL_SOLES'],
 

    #['_9_cut_up_NUM_MESES_ANTERIORES_CON_IMPORTE_PROT_CONSECUTIVOS', 'IMPORTE_TRIB_REGISTRADO_MAYOR_A_15MIL_SOLES'],
    
    # 
    ['TIENE_MAYORIGUAL_1.05_U1M_2M_OVER_U12M_U11M_RIESGO_DIRECTO_CIERRE_MES', 'IMPORTE_IMPAG_REGISTRADO_MAYOR_A_10MIL_SOLES'],
    
    ['PROTESTOS_REGISTRADO_MAYOR_A_5MIL_SOLES', 'NO_TIENE_GAR_AUTO_SF', '_1_cut_up_MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x'],
 #['TIENE_REACTIVA_1_2_CON_OTRAS_EF', '_2_cut_up_MAX_DIAS_ATRASO_SFSF_ULT_RCC'],
 #['cut_1_TIENE_AL_MENOS_1_TIPO_CASHIN_ACTIVO_U3M', '_aux_NO_TIENE_CASHOUT_OPERACIONES_MESA__HACE_12M', '_3_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF'],
 #['TIENE_DISMINUCION_10PERCENT_A_MAS_CANT_TRX_PROM_U6M_U12M', '_3_cut_up_BUCKET'],
    
 ['TIENE_10PERCENT_O_MAS_SEI_VPC_FEVE_DRO', '_4_cut_up_NUM_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M', '_3_cut_up_NUM_MESES_ANTERIORES_CON_IMPORTE_IMPAG_CONSECUTIVOS'],
 ['TIENE_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO_MAYOR_1', '_1_cut_up_NUM_DOCS_IMPAG_U6M', 'TIENE_REACTIVA_CON_OTRA_EF'],
 ['TIENE_6_O_MAS_MESES_CON_RIESGO_INDIRECTO_CIERRE_MES_EN_CERO', 'FEVE_GARANTIZAR'],
 ['_1.4_cut_up_APLANCAMIENTO_PATRIMONIAL_VAR_CEF_SITUACION', 'TIENE_MAS_DE_3MESES_SIN_LINEA_VIGENTE'],


]
len(lista_alertas_orden)


29

In [21]:
for orden in range(len(lista_alertas_orden)):
    alerta = lista_alertas_orden[orden]
    
    orden_interno = 1
    for variable in alerta:
        orden_alerta = 'A{}_{}'.format(orden + 1, orden_interno)
        data[orden_alerta] = data[variable]
        #print(">> ", orden  +1 ,' : ',orden_alerta, '  --  ', variable)
        
        orden_interno += 1
        
def ambas_1(a ,b):
    return int(a >= 1 and b >= 1)

def todas_1(a ,b, c):
    return int(a >= 1 and b >= 1 and c >= 1)

def megas_1(a ,b, c, d):
    return int(a >= 1 and b >= 1 and c >= 1 and d >= 1)

alerts_ = []
for i in range(len(lista_alertas_orden)):
    con_alerta = 'A'+str(i+1)
    alerts_.append(con_alerta)
    #print("-> ", con_alerta, lista_alertas_orden[i])
    if len(lista_alertas_orden[i]) == 2:
        data[con_alerta] =  data[[con_alerta + '_1', con_alerta + '_2']].apply(
            lambda _: ambas_1(_[0], _[1]), axis=1
        )
    elif len(lista_alertas_orden[i]) == 3:
        data[con_alerta] =  data[[con_alerta + '_1', con_alerta + '_2', con_alerta + '_3']].apply(
            lambda _: todas_1(_[0], _[1], _[2]), axis=1
        )
    elif len(lista_alertas_orden[i]) == 4:
        data[con_alerta] =  data[[con_alerta + '_1', con_alerta + '_2', con_alerta + '_3',con_alerta + '_4']].apply(
            lambda _: megas_1(_[0], _[1], _[2], _[3]), axis=1
        )
    else:
        pass
    
#cols_alerta = sorted(list({col.split('_')[0] for col in data.columns if col.startswith("A")}))
data['FLG_ALERTA'] = data[alerts_].apply(lambda _: 1 if sum(_) > 0 else 0, axis=1)

In [22]:
data['PERIODO'].value_counts()

202102    605
202101    602
Name: PERIODO, dtype: int64

In [23]:
con_alerta = data[
    (data['FLG_ALERTA'] >= 1) & (data[col_target] >= 1) 
]
con_alerta['PERIODO'].value_counts()

202102    35
202101    34
Name: PERIODO, dtype: int64

In [24]:
ya_alertados = set(list(con_alerta['COD_UNICO'].unique()))
len(ya_alertados)

56

In [28]:
dicc_show = {
    'var_1': [],
    'var_2': [],
    'var_3': [],
    'alertados': [],
    'correctos': [],
    'nuevos': [],
    'efectividad': [],
}
grupo_alerta = {}
cols_usada = []
lista_coinciden_valida = []
lista_alertados = []
listado_por_alerta = []

for row in select.sort_values(by=['SCORE'], ascending=False).iterrows():
    cols_1 = row[1][0]
    cols_2 = row[1][1]
    cols_3 = row[1][2]
    cantidad = row[1][3]
    correctos = row[1][4]
    score_feve = float(row[1][5])
    
    _all = cols_1 + ' ' + cols_2  if pd.isnull(cols_3) else cols_1 + ' ' + cols_2 + ' ' + cols_3
    
    if not _all.endswith('U6M') : 
       #and cols_2 not in cols_usada and cols_3 not in cols_usada \
       #and 'suspendido' not in _all and 'cancelado' not in _all and 'norte_exp' not in _all  and 'vencid' not in _all and 'cef' not in _all.lower():
        #print(_all)
        try: 
            if pd.isnull(cols_3):
                data_que_cumple = data[
                    (data[cols_1] >= 1) & (data[cols_2] >= 1)
                ]
            else:
                cols_usada.append(cols_3)
                data_que_cumple = data[
                    (data[cols_1] >= 1) & (data[cols_2] >= 1) &  (data[cols_3] >= 1)
                ]
                
            alertados = set(data_que_cumple['COD_UNICO'])
            listado_por_alerta.append(list(alertados))
            cols_usada.append(cols_1)
            cols_usada.append(cols_2)
            
            
            bien_alertados = alertados & set(malos['COD_UNICO'])
            alertados_nuevos = bien_alertados - ya_alertados   #####################
            
            #print(data_que_cumple.shape, bien_alertados)
            if len(bien_alertados) > 4  and len(bien_alertados) / len(alertados) >= 0.24 and len(alertados_nuevos) > 0: #cañito
                lista_alertados.extend(alertados)
                lista_coinciden_valida.extend(bien_alertados)
                dicc_show['var_1'].append(cols_1)
                dicc_show['var_2'].append(cols_2)
                dicc_show['var_3'].append(cols_3)
                dicc_show['alertados'].append(len(set(alertados)))
                dicc_show['correctos'].append(len(set(bien_alertados)))
                dicc_show['nuevos'].append(len(alertados_nuevos))
                dicc_show['efectividad'].append(len(set(bien_alertados)) / len(set(alertados)))
            
                #print(dicc_show['var_1'][-1], '|||', dicc_show['var_2'][-1],  '|||', dicc_show['var_3'][-1], 'de: ',
                #      dicc_show['alertados'][-1], 'coinciden : ', dicc_show['correctos'][-1], dicc_show['efectividad'][-1]
                #)
                grupo_alerta[cols_1 + '|'+ cols_2 + '|'+ str(cols_3)] = 'de: {} coinciden: {}'.format(len(alertados), len(bien_alertados))
            
            if len(set(lista_coinciden_valida)) / len(set(malos['COD_UNICO'])) > 0.9999:
                print("="*100)
                print("De {} alertó bien a: {}".format(
                    len(set(lista_alertados)), len(set(lista_coinciden_valida))
                ))
                
                clientes_detectados_ok = set(lista_coinciden_valida)
                clientes_no_alertar = set(lista_alertados) - set(clientes_detectados_ok)
                
                dicc_cliente_no_alertar = {}
                print("ANALIZANDO CLIENTES ")
                for cliente in clientes_no_alertar:
                    aparazco = 0
                    for lista_de_alerta in listado_por_alerta:
                        if cliente in lista_de_alerta:
                            aparazco += 1
                    dicc_cliente_no_alertar[cliente] = aparazco
                    
                dicc_cliente_ok = {}
                print("ANALIZANDO OK ")
                for cliente in clientes_detectados_ok:
                    aparazco = 0
                    for lista_de_alerta in listado_por_alerta:
                        if cliente in lista_de_alerta:
                            aparazco += 1
                    dicc_cliente_ok[cliente] = aparazco
                break
        
        except Exception as e:
            pass
            #print(">"*50, str(e))
        #    print(str(e))
        #    #coinciden_feve_duro

print(
    "SOLO CUBRE: ", len(set(lista_coinciden_valida)) / len(set(malos['COD_UNICO'])), len(set(lista_coinciden_valida)), 
     len(set(malos['COD_UNICO']))
)

SOLO CUBRE:  0.948051948051948 73 77


In [29]:
struct = pd.DataFrame(dicc_show)
struct['mult'] = struct[['var_1', 'var_2', 'var_3']].apply(lambda _: _[0] + '----' + _[1] + '----' + str(_[2]), axis=1)

In [30]:
pd.set_option('display.max_rows', 20000)
pd.set_option('display.max_columns', 20000)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.width', 10000)

struct[
    (struct['efectividad'] > 0.3) & (struct['alertados'] < 30) & (struct['nuevos'] > 2)
].sort_values(
    by=['nuevos'], ascending=False
)[
    ['efectividad', 'alertados', 'correctos', 'nuevos', 'mult']
]

,efectividad,alertados,correctos,nuevos,mult
618,0.461538,13,6,4,TIENE_DISMINUCION_PAGO_PROVEEDORES_30PERCENT_A_MAS_U7M----_2_cut_up_NUM_DOCS_PROT_U12M----nan
630,0.461538,13,6,4,TIENE_DISMINUCION_PAGO_PROVEEDORES_30PERCENT_A_MAS_U7M----_2_cut_up_NUM_DOCS_PROT_U12M----_aux_TIENE_CASHOUT_PAGOS_MASIVOS__U12M
780,0.461538,13,6,4,TIENE_DISMINUCION_PAGO_PROVEEDORES_30PERCENT_A_MAS_U7M----_1_cut_up_NUM_DOCS_PROT_U12M----_2_cut_up_NUM_BANCOS_PRIN_DEUDASF_ULT_RCC
1353,0.312500,16,5,4,TIENE_DISMINUCION_PAGO_PROVEEDORES_30PERCENT_A_MAS_U7M----_1_cut_up_NUM_DOCS_PROT_U6M----_0.6_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC
1370,0.357143,14,5,4,TIENE_DISMINUCION_PAGO_PROVEEDORES_30PERCENT_A_MAS_U7M----_0.4_cut_down_PERCENT_GAR_HIPO----_1_cut_up_NUM_DOCS_PROT_U12M
1472,0.375000,16,6,4,TIENE_DISMINUCION_PAGO_PROVEEDORES_30PERCENT_A_MAS_U7M----_1_cut_up_NUM_DOCS_PROT_U12M----nan
1478,0.333333,15,5,4,TIENE_DISMINUCION_PAGO_PROVEEDORES_30PERCENT_A_MAS_U7M----_1_cut_up_NUM_DOCS_PROT_U12M----_0.5_cut_down_minimo_percent_auto_hipo_pren_u12m
687,0.344828,29,10,3,_8_cut_up__int_FM_category_quartil----_5_cut_up_NUM_DOCS_IMPAG_U1M----nan
688,0.344828,29,10,3,_8_cut_up__int_FM_category_quartil----_5_cut_up_NUM_DOCS_IMPAG_U1M----_-34_cut_up_DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT
790,0.357143,28,10,3,_0_cut_down_CAPITAL_TRABAJO_CEF_SITUACION----_aux_TIENE_TRX_PAGO_DE_SERVICIOS__CASH_OUT_U3M----nan
